Code of registration of Ca2+ imaging time stacks before and after treatments.

@uthor: Raju Tomer.

In [2]:
import monnet_utils as tl
import os
import SimpleITK as sitk
import numpy as np
import skimage.external.tifffile as tff

In [4]:
###Read file names
main_data_dir = r'E:\Raju\MoNNet'

data_dir01 = os.path.join(main_data_dir, r'BT')
data_dir02 = os.path.join(main_data_dir, r'AT')

def gen_fn_in_dir(data_dir):
    dict_in_fn={}
    i=0
    for root1, dirs1, files1 in os.walk(data_dir):
        files1.sort()
        for fn in files1:
            if (fn.endswith('.tif') and ('MAX_' in fn)):
                print('Processing ', fn)
                dict_in_fn[i] = fn
                i=i+1
    return dict_in_fn

dict_in_fn01 = gen_fn_in_dir(data_dir01)
dict_in_fn02 = gen_fn_in_dir(data_dir02)

for key in dict_in_fn01.keys():
    print(key, dict_in_fn01[key])
    print(key, dict_in_fn02[key])


Processing  MAX_21-11-12_D1WB_bs_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D2WB_bs_Red4x_moco-crp.tif
Processing  MAX_21-11-12_D3WA_bs_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D4WC_Red4x_bs_moco_crp.tif
Processing  MAX_21-11-12_D5WB_bs_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D6WC_bs_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D1WB_1uMPicro_Red4x.tif
Processing  MAX_21-11-12_D2WB_1uMPicro_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D3WA_1uMPicro_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D4WC_100nMPicro_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D5WB_100nMPicro_Red4x_moco_crp.tif
Processing  MAX_21-11-12_D6WC_100nMPicro_Red4x_moco_crp.tif
0 MAX_21-11-12_D1WB_bs_Red4x_moco_crp.tif
0 MAX_21-11-12_D1WB_1uMPicro_Red4x.tif
1 MAX_21-11-12_D2WB_bs_Red4x_moco-crp.tif
1 MAX_21-11-12_D2WB_1uMPicro_Red4x_moco_crp.tif
2 MAX_21-11-12_D3WA_bs_Red4x_moco_crp.tif
2 MAX_21-11-12_D3WA_1uMPicro_Red4x_moco_crp.tif
3 MAX_21-11-12_D4WC_Red4x_bs_moco_crp.tif
3 MAX_21-11-12_D4WC_100nMPicro_Red4x_moc

In [5]:
#### Align Max proj images of before and after
out_dir = r'E:\Raju\MoNNet\AT_aln'
for key in dict_in_fn01.keys():
    print(key)
    fn01 = dict_in_fn01[key]
    fn02 = dict_in_fn02[key]
    im_max01 = sitk.ReadImage(os.path.join(data_dir01, fn01), sitk.sitkFloat32)
    im_max02 = sitk.ReadImage(os.path.join(data_dir02, fn02), sitk.sitkFloat32)
    
    final_transform = tl.align_2d(im_max01, im_max02, aff=True)
    print(final_transform)
    im_max02_aln = sitk.Resample(im_max02, im_max01, final_transform, sitk.sitkLinear, 0.0, im_max02.GetPixelID())

    im_max01_aln = np.copy(sitk.GetArrayFromImage(im_max01))
    im_max02_aln = np.copy(sitk.GetArrayFromImage(im_max02_aln))
    im_max01_aln = im_max01_aln.astype('uint16')
    im_max02_aln = im_max02_aln.astype('uint16')
    
    im_max_combo = np.zeros((5, im_max01_aln.shape[0], im_max01_aln.shape[1]), dtype='uint16')
    print(im_max_combo.shape)
    im_max_combo[0,:,:] = im_max01_aln
    im_max_combo[1,:,:] = im_max02_aln
    
    out_fn = os.path.join(out_dir, 'im_max_combo_key' + str(key) + '.tif')
    tff.imsave(out_fn, im_max_combo)


0
Using Affine
Final metric value: -0.4661451861181417
Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Gradient magnitude tolerance met after 19 iterations. Gradient magnitude (9.52944e-05) is less than gradient magnitude tolerance (0.0001).
itk::simple::Transform
 CompositeTransform (00000000070C4D20)
   RTTI typeinfo:   class itk::CompositeTransform<double,2>
   Reference Count: 1
   Modified Time: 385294
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   AffineTransform (000000000CEB62D0)
     RTTI typeinfo:   class itk::AffineTransform<double,2>
     Reference Count: 1
     Modified Time: 385285
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.996154 -0.0659095 
       0.0454135 0.961998 
     Offset: [6.65409, 1.3826]
     Center: [205, 116]
     Translation: [-1.77985, 6.28413]
     Inverse: 
       1.00074 0.0685635 
       -0.0472422 1.03627 
     Singula

In [ ]:
### After manual validation, apply all those parameters to entire stacks
out_dir = r'E:\Raju\MoNNet\AT_aln'

for key in dict_in_fn01.keys():
    print(key)
    fn01 = dict_in_fn01[key]
    fn02 = dict_in_fn02[key]
    im_max01 = sitk.ReadImage(os.path.join(data_dir01, fn01), sitk.sitkFloat32)
    im_max02 = sitk.ReadImage(os.path.join(data_dir02, fn02), sitk.sitkFloat32)
    fn02 = fn02.replace('MAX_','')
    #print(fn02)
    print('Stack reading')
    im_max02_stack = tff.imread(os.path.join(data_dir02, fn02))
    
    print('Stack read')
    final_transform = tl.align_2d(im_max01, im_max02, aff=True)
    #print(final_transform)
    im_max02_aln = sitk.Resample(im_max02, im_max01, final_transform, sitk.sitkLinear, 0.0, im_max02.GetPixelID())
    t_im = np.copy(sitk.GetArrayFromImage(im_max01))
    im_max02_stack_aln = np.zeros((im_max02_stack.shape[0], t_im.shape[0], t_im.shape[1]), dtype='uint16')
    
    for ind in range(im_max02_stack.shape[0]):
        #print(ind)
        im_tmp = im_max02_stack[ind,:,:]
        im_tmp = sitk.GetImageFromArray(im_tmp)
        im_tmp = sitk.Resample(im_tmp, im_max01, final_transform, sitk.sitkLinear, 0.0, im_max02.GetPixelID())
        im_tmp = np.copy(sitk.GetArrayFromImage(im_tmp))
        im_max02_stack_aln[ind,:,:] = im_tmp.astype('uint16')
    
    im_max01_aln = np.copy(sitk.GetArrayFromImage(im_max01))
    im_max02_aln = np.copy(sitk.GetArrayFromImage(im_max02_aln))
    im_max01_aln = im_max01_aln.astype('uint16')
    im_max02_aln = im_max02_aln.astype('uint16')
    
    im_max_combo = np.zeros((5, im_max01_aln.shape[0], im_max01_aln.shape[1]), dtype='uint16')
    print(im_max_combo.shape)
    im_max_combo[0,:,:] = im_max01_aln
    im_max_combo[1,:,:] = im_max02_aln
    
    out_fn = os.path.join(out_dir, 'im_max_combo_key' + str(key) + '.tif')
    tff.imsave(out_fn, im_max_combo)
    fn02 = fn02.rsplit(".", 1)[0]
    out_fn = os.path.join(out_dir, fn02 + '_aln.tif')
    tff.imsave(out_fn, im_max02_stack_aln)
    print('Done: ', out_fn)